In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import re

In [ ]:
feature_ID_list = pd.read_csv('./Feature ID list for shock(MIMIC-4).csv')
feature_name = feature_ID_list['Feature name'].tolist()
feature_ID_list.set_index('Feature name', inplace=True)
urine_id_ls = feature_ID_list['outputevents'][feature_name[27]]
urine_id_ls = [int(x) for x in urine_id_ls.split(',')]

In [ ]:
outputevents = pd.read_csv('./outputevents.csv')
patients = pd.read_csv('./icustays.csv')
#patients.drop(columns=['Unnamed: 0'], inplace=True)
#patients.drop(columns=['ICUSTAY_ID'], inplace=True)

outputevents = pd.merge(outputevents, patients, on=['subject_id','hadm_id','stay_id'], how='left')
#data = data[data['los']>1]
outputevents.reset_index(drop=True, inplace=True)

condition1 = (outputevents['intime'] <= outputevents['charttime'])
condition2 = (outputevents['outtime'] >= outputevents['charttime'])

outputevents = outputevents[condition1 & condition2]

urine = outputevents[outputevents['itemid'].isin(urine_id_ls)]
urine = urine[['subject_id','hadm_id','stay_id', 'intime', 'outtime', 'charttime','value']]
urine.rename(columns={'value':'Urine Output'}, inplace=True)
condition1 = urine['Urine Output']>=0 
urine = urine[condition1]
urine.reset_index(drop=True, inplace=True)

In [ ]:
urine['charttime'] = pd.to_datetime(urine['charttime'])
urine['outtime'] = pd.to_datetime(urine['outtime'])
urine['intime'] = pd.to_datetime(urine['intime'])

In [ ]:
def resampling_func_urine(x):
    column = ['subject_id','hadm_id','stay_id', 'time', 'Urine Output']

    feature = 'Urine Output'

    x.reset_index(drop=True, inplace=True)

    date_range = pd.date_range(x['intime'][0], x['outtime'][0], freq='10T', inclusive='right')
    new_df = pd.DataFrame(index=date_range, columns=column)

    new_df['subject_id'] = x['subject_id'][0]
    new_df['hadm_id'] = x['hadm_id'][0]
    new_df['stay_id'] = x['stay_id'][0]
    new_df['time'] = new_df.index
    new_df['Urine Output'] = 0

    for i in range(len(date_range)):
        if i == 0:
            condition1 = date_range[i]>=x['charttime']
            sa = x[condition1]
            new_df[feature][i] = sa['Urine Output'].sum()

        else :
            condition2 = date_range[i-1]<x['charttime']
            condition3 = date_range[i]>=x['charttime']
            sa = x[(condition2)&(condition3)]
            new_df[feature][i] = sa['Urine Output'].sum()

    return new_df

result_urine = urine.groupby('stay_id').apply(resampling_func_urine)
result_urine = result_urine.reset_index(drop=True)
print('---------------------------------------------------------------------------------')
result_urine.to_csv('./Data preprocessed 4/10min Urine output.csv', index=False)

In [ ]:
result_urine